In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

with open("setting.yaml") as yaml_file:

    SETTING = yaml.load(yaml_file)

PATH = make_path_dict(SETTING)

In [ ]:
target_x_sample = pd.read_csv(PATH["target_x_sample.tsv"], sep="\t", index_col=0)

gene_set_x_sample = pd.read_csv(PATH["gene_set_x_sample.tsv"], sep="\t", index_col=0)

In [ ]:
figure_name_data_name_indices = {
    "Figure 0": {
        "Stemness and EMT Signatures": [
            "Cancer Stem Cell",
            "BENPORATH_ES_CORE_NINE",
            "BENPORATH_ES_WITH_H3K27ME3",
            "HOEBEKE_LYMPHOID_STEM_CELL_UP",
            "IVANOVA_HEMATOPOIESIS_STEM_CELL_AND_PROGENITOR",
            "MIKKELSEN_IPS_LCP_WITH_H3K4ME3",
            "MIKKELSEN_IPS_WITH_HCP_H3K27ME3",
            "GOTZMANN_EPITHELIAL_TO_MESENCHYMAL_TRANSITION_UP",
        ],
        "Oncogenic Signatures": [
            "HALLMARK_NOTCH_SIGNALING",
            "HALLMARK_TGF_BETA_SIGNALING",
            "HALLMARK_E2F_TARGETS",
            "HALLMARK_WNT_BETA_CATENIN_SIGNALING",
            "BIOCARTA_WNT_PATHWAY",
            "PRC2_EZH2_UP.V1_UP",
            "E2F3_UP.V1_UP",
        ],
    },
    "Figure 1": {
        "NFkB Signatures": [
            "HINATA_NFKB_TARGETS_KERATINOCYTE_UP",
            "TIAN_TNF_SIGNALING_VIA_NFKB",
        ],
        "Differentiation Signatures": [
            "RODRIGUES_THYROID_CARCINOMA_POORLY_DIFFERENTIATED_UP",
            "MA_MYELOID_DIFFERENTIATION_UP",
            "ADDYA_ERYTHROID_DIFFERENTIATION_BY_HEMIN",
        ],
    },
}

In [ ]:
for target_name, target_values in target_x_sample.iterrows():

    output_directory_path = "{}/{}".format(PATH["summary/"], target_name)

    ccal.establish_path(output_directory_path, "directory")

    target_values = target_values[target_values != -1]

    for figure_name, data_name_indices in figure_name_data_name_indices.items():

        data_dicts = {}

        score_moe_p_value_fdr_dicts = {}

        for data_name, indices in data_name_indices.items():

            data_dicts[data_name] = {
                "df": gene_set_x_sample.loc[indices],
                "type": "continuous",
            }

            score_moe_p_value_fdr_dicts[data_name] = pd.read_csv(
                "{}/{}/all.tsv".format(
                    PATH["differentially_expressed_gene_set/"], target_name
                ),
                sep="\t",
                index_col=0,
            )

        ccal.make_summary_match_panel(
            target_values,
            data_dicts,
            score_moe_p_value_fdr_dicts,
            target_type="binary",
            plot_std=SETTING["plot_std"],
            title=dict(text="{}<br>{}".format(target_name, figure_name)),
            xaxis_kwargs=dict(),
            html_file_path="{}/{}.html".format(output_directory_path, figure_name),
        )